In [36]:
import pandas as pd
df = pd.read_excel('ceshi.xlsx')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10000 non-null  int64 
 1   label       10000 non-null  int64 
 2   clean_text  10000 non-null  object
dtypes: int64(2), object(1)
memory usage: 234.5+ KB


In [38]:
#Import feature engineering modules and test_train_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.svm import SVC

#Import modules to calculate accuracy and confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [48]:
text_clf = Pipeline([('vect', TfidfVectorizer()),
                     ('clf', SVC())])

# 设置超参数组合
tuned_parameters = {
    'vect__ngram_range': [(1, 2), (1, 3)],
    'clf__tol': [1e-1, 1e-2, 1e-3]
}

In [49]:
x_train, x_test, y_train, y_test = train_test_split(df['clean_text'], df['label'],
                                               test_size = 0.3, shuffle=True)

In [50]:
clf = GridSearchCV(text_clf, tuned_parameters, cv=10) # 输入超参数
clf.fit(x_train, y_train) #开始训练



GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf', SVC())]),
             param_grid={'clf__tol': [0.1, 0.01, 0.001],
                         'vect__ngram_range': [(1, 2), (1, 3)]})

In [51]:
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

Best Score:  0.7747142857142857
Best Params:  {'clf__tol': 0.001, 'vect__ngram_range': (1, 2)}


In [52]:
print(classification_report(y_test, clf.predict(x_test), digits=4))
print(accuracy_score(y_test, clf.predict(x_test)))
print(confusion_matrix(y_test, clf.predict(x_test)))

              precision    recall  f1-score   support

          -1     0.7548    0.8154    0.7839      1533
           1     0.7894    0.7232    0.7549      1467

    accuracy                         0.7703      3000
   macro avg     0.7721    0.7693    0.7694      3000
weighted avg     0.7718    0.7703    0.7697      3000

0.7703333333333333
[[1250  283]
 [ 406 1061]]


In [53]:
optimal_SVM = clf.best_estimator_

In [57]:
text = ['i hate you, you are a bad man, i do not like you,get out', 'today is a nice day let us go working']
optimal_SVM.predict(text)

array([-1,  1])